1)from 2)where 3)select 4)orderby<br>
a.Support for the LIMIT clause.<br>
b. Support for the DELETE statement.

In [3]:
import pandas as pd
import operator
# import warnings
# warnings.filterwarnings('ignore')

In [4]:
def common(t1,t2,a,b):
    T =set(list(a.columns)) & set(list(b.columns))
    T1=list(T)
    a1,b1 =list(a.columns),list(b.columns)
        # if any common arrtibute bw 2 tables
        # appending the common arrtibute with table name.
    for i in range(len(T1)):
        l=a1.index(T1[i])#table 1
        k=b1.index(T1[i])#table 2
        a1[l]=a1[l].replace(a1[l],t1+str(a1[l]))
        b1[l]=b1[l].replace(b1[l],t2+str(b1[k]))
        a.rename(columns ={"a[l]":"a1[l]"}, inplace = True)
        b.rename(columns ={"b[l]":"b1[l]"}, inplace = True)
    return(a1,b1)

In [5]:
# CROSS JOIN 
def cartesianjoin(t1,t2,a,b):
    k= 0
    # to find common arrtibutes in both the files
    T =set(list(a.columns)) & set(list(b.columns))
    T1=list(T)
    if T1!=[]:
        a1,b1=common(t1,t2,a,b)#if any common arrtibute
        df = pd.DataFrame(columns=a1+b1)
        for i in range(a.shape[0]):
            for j in range(b.shape[0]):
                df.loc[k] = list(a.loc[i])+list(b.loc[j])
                k+=1
                
    else:
        df = pd.DataFrame(columns=list(a.columns)+list(b.columns))
        for i in range(a.shape[0]):
            for j in range(b.shape[0]):
                df.loc[k] = list(a.loc[i])+list(b.loc[j])
                k+=1
    return(df)

In [6]:
# Natural join
def naturaljoin(t1,t2,a,b):
    k=0
    # to find common arrtibutes in both the files
    T =set(list(a.columns)) & set(list(b.columns))
    if T=={}:
        return cartesianjoin(t1,t2,a,b)
    df = pd.DataFrame(columns=list(a.columns)+list(set(b.columns)-T))
    for i in range(a.shape[0]):
        for j in range(b.shape[0]):
            if set(a.loc[i,list(T)]) == set(b.loc[j,list(T)]):
                df.loc[k] = list(a.loc[i])+list(b.loc[j,list(set(b.columns)-T)])
                k+=1
    return(df)

In [7]:
def FromClause(s2):
    a = []
    j=0
    a.append(pd.read_csv(s2[0]))
    i=1
    while i<len(s2):
        if s2[i] == "NATURAL":
            print("NaturalJoin bewteen\t"+str(s2[i-1])+"\t"+str(s2[i+2]))
            t1,t2 = s2[i-1][:-3],s2[i+2][:-3]
            a.append(pd.read_csv(s2[i+2]))
            a[j+1]=naturaljoin(t1,t2,a[j],a[j+1])
            j+=1
            i+=3
        
        else:
            print("CartesianJoin bewteen\t"+str(s2[i-1])+"\t"+str(s2[i]))
            t1,t2 = s2[i-1][:-3],s2[i][:-3]
            a.append(pd.read_csv(s2[i]))
            a[j+1]=cartesianjoin(t1,t2,a[j],a[j+1])
            j+=1
            i+=1
    return(a[-1])

In [8]:
def where(df,s3):
    if s3[0] in list(df.columns):
        tmp=s3[0]
    else:
        for i in range(len(s3)):
            if '.' in s3[i]:
                tmp=s3[i].index('.')#index of the string with "."
                tmp =s3[i][tmp+1:]#string after "."
                break
            else:
                tmp='Data.'+s3[0]
                break
    try:
        s3[2] = float(s3[2])
    except:
        pass
    
    if s6 == 0:
        if(s3[1]=='>='):
            return df[operator.__ge__(df[tmp],s3[2])]
        elif(s3[1]=='>'):
            return df[operator.__gt__(df[tmp],s3[2])]
        elif(s3[1]=='<='):
            return df[operator.__le__(df[tmp],s3[2])]
        elif(s3[1]=='<'):
            return df[operator.__lt__(df[tmp],s3[2])]
        elif(s3[1]=='='):
            return df[operator.__eq__(df[tmp],s3[2])]
        else:
            return df[operator.__ne__(df[tmp],s3[2])]
    else: #for delete
        if(s3[1]=='>='):
            return df[operator.__lt__(df[tmp],s3[2])]
        elif(s3[1]=='>'):
            return df[operator.__le__(df[tmp],s3[2])]
        elif(s3[1]=='<='):
            return df[operator.__gt__(df[tmp],s3[2])]
        elif(s3[1]=='<'):
            return df[operator.__ge__(df[tmp],s3[2])]
        elif(s3[1]=='='):
            return df[operator.__ne__(df[tmp],s3[2])]
        else:
            return df[operator.__eq__(df[tmp],s3[2])]


In [9]:
def select(df,s1):
    if s1[0]=='*':
        return df
    else:
        for i in range(len(s1)):
            if s1[i] not in list(df.columns):
                tmp=s1[i].index('.')
                s1[i]=s1[i][tmp+1:]
        return df[s1]

In [10]:
def merge(df, l, m, r,cmp,col):
    idx = list(df.index) # storing index values in a list
    n1 = m - l + 1
    n2 = r - m
    idx1 = idx[:n1]
    idx2 = idx[n1:n1+n2]
    L = pd.DataFrame(columns = df.columns)
    R = pd.DataFrame(columns = df.columns)
    # Copy data to temp arrays L[] and R[]
    for i in range(len(idx1)):
        L.loc[idx1[i]] = df.iloc[l + i]
 
    for j in range(len(idx2)):
        R.loc[idx2[j]] = df.iloc[m + 1 + j]
 
    # Merge the temp arrays back into arr[l..r]
    i = j = 0    
    k = l    
 
    while i < n1 and j < n2:
        if cmp =='<=':
            if L.iloc[i,col] <=  R.iloc[j,col]:
                df.loc[idx[k]] = L.loc[idx1[i]]
                i += 1
            else:
                df.loc[idx[k]] = R.loc[idx2[j]]
                j += 1
            k += 1
        else:
            if L.iloc[i,col] >=  R.iloc[j,col]:
                df.loc[idx[k]] = L.loc[idx1[i]]
                i += 1
            else:
                df.loc[idx[k]] = R.loc[idx2[j]]
                j += 1
            k += 1
 
    # Copy the remaining elements of L[], if there are any
    while i < n1:
        df.loc[idx[k]] = L.loc[idx1[i]]
        i += 1
        k += 1
 
    # Copy the remaining elements of R[], if there are any
    while j < n2:
        df.loc[idx[k]] = R.loc[idx2[j]]
        j += 1
        k += 1
    return df
def mergeSort(df, l, r,cmp,col):
    if l < r:
        m = l+(r-l)//2
        df=mergeSort(df, l, m,cmp,col)
        df=mergeSort(df, m+1, r,cmp,col)
        df=merge(df, l, m, r,cmp,col)
    return df

In [11]:
def orderby(df,s4):
    if s4[0] in list(df.columns):
        col=0
        if s4[1]=='ASC':
            cmp='<='
            return mergeSort(df,0,df.shape[0]-1,cmp,col)
        else:
            cmp='>='
            return mergeSort(df,0,df.shape[0]-1,cmp,col)
    else:
        tmp = s4[0].index('.')
        s4[0]=s4[0][tmp+1:]
        col=0
        if s4[1]=='ASC':
            cmp='<='
            return mergeSort(df,0,df.shape[0]-1,cmp,col)
        else:
            cmp='>='
            return mergeSort(df,0,df.shape[0]-1,cmp,col)

In [12]:
def limit(df,l):
    # return df.iloc[:l]
    return df.head(l)

In [13]:
# SELECT * FROM Data.csv;
# SELECT name, Age FROM Data.csv;

# SELECT * FROM Data.csv WHERE name = sai;
# SELECT name, Age FROM Data.csv WHERE name = sai;

# SELECT name, Age FROM Data.csv WHERE name = sai ORDER BY name DESC;
# SELECT * FROM Data.csv WHERE name = sai ORDER BY name DESC;

# SELECT Data.name, student.subject FROM Data.csv, student.csv WHERE Data.Age >= 25 ORDER BY Data.name DESC;
# SELECT Data.name, student.subject FROM Data.csv, student.csv WHERE Data.Age >= 25 ORDER BY Data.name ASC;
# SELECT Data.name, student.subject FROM Data.csv NATURAL JOIN student.csv WHERE Data.Age >= 25 ORDER BY Data.balance DESC;

# SELECT Data.name, student.subject FROM Data.csv, student.csv WHERE Data.Age >= 25 ORDER BY Data.name ASC LIMIT 4;

# DELETE FROM Data.csv, student.csv WHERE Data.Age >= 25 ORDER BY Data.name ASC LIMIT 4;
# DELETE FROM Data.csv, student.csv WHERE Data.Age >= 25 ORDER BY Data.name ASC;
# DELETE FROM Data.csv, student.csv;


# SELECT Data.name,student.class FROM Data.csv NATURAL JOIN student.csv;
# SELECT * FROM Data.csv LIMIT 5;
# SELECT * FROM Data.csv ORDER BY Data.name;
# DELETE FROM Data.csv WHERE Data.Age >= 50;





In [53]:
query = input("Enter the Query you want Where key words should be in Capital:\n")
q = query[:-1]
q = q.replace(","," ")
q = q.replace(", ","")


# s1 is for select
# s6 is for delet
s6 = 0
s1 = ""
if 'SELECT' in q:
    s1=q[q.index('SELECT')+7:q.index('FROM')-1].split()
else: #if delete is there in query ,just do the opposite of select and set s6 =1
    s6=1

# s2 is for from
if 'WHERE' in q:
    s2=q[q.index('FROM')+5:q.index('WHERE')-1].split()
elif 'ORDER bY' in q:
    s2=q[q.index('FROM')+5:q.index('ORDER BY')-1].split()
elif 'LIMIT' in q:
    s2=q[q.index('FROM')+5:q.index('LIMIT')-1].split()
else:
    s2=q[q.index('FROM')+5:].split()


# s3 is for where
s3=""
if 'WHERE' in q:
    if 'ORDER BY' in q:
        s3=q[q.index('WHERE')+6:q.index('ORDER BY')-1].split()
    elif 'LIMIT' in q:
        s3=q[q.index('WHERE')+6:q.index('Limit ')-1].split()
    else:
        s3=q[q.index('WHERE')+6:].split()


# s4 for order by
s4=""
if 'BY' in q:
    if 'LIMIT' in q:
        s4=q[q.index('BY')+3:q.index('LIMIT')-1].split()
    else:
        s4=q[q.index('BY')+3:].split()

# s5 for limit
s5=""
if "LIMIT" in q:
    s5 = q[q.index("LIMIT")+6:].split()

In [54]:
# Main function
df = FromClause(s2)
if s3!="":
    df = where(df,s3)
elif s6==1:
    df =df.head(0)

if s1!="":
    df=select(df,s1)

if s4!="":
    df = orderby(df,s4)

if s5!="":
    df = limit(df,int(s5[0]))

print(df)
df.to_csv("output.csv")

CartesianJoin bewteen	Data.csv	student.csv
     Data.name subject
0  Ghatotkacha    unix
1  Ghatotkacha      ml
2  Ghatotkacha      ml
3  Ghatotkacha    unix
